In [ ]:
!pip install -q -U transformers bitsandbytes datasets fsspec

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset

In [ ]:
# Use a pipeline as a high-level helper
# from transformers import pipeline

# messages = [
#     {"role": "user", "content": "Who are you?"},
# ]
# pipe = pipeline("text-generation", model="meta-llama/Meta-Llama-3-8B-Instruct")
# pipe(messages)

In [ ]:
# Load the 7b llama model
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, # quantize the weights to 4-bit
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

# Load model
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

dataset_id = "nvidia/HelpSteer2"
dataset = load_dataset(dataset_id)

dataset

README.md:   0%|          | 0.00/25.0k [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.jsonl.gz:   0%|          | 0.00/582k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20324 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1038 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response', 'helpfulness', 'correctness', 'coherence', 'complexity', 'verbosity'],
        num_rows: 20324
    })
    validation: Dataset({
        features: ['prompt', 'response', 'helpfulness', 'correctness', 'coherence', 'complexity', 'verbosity'],
        num_rows: 1038
    })
})

In [ ]:
dataset["train"][11]["prompt"]

'Please, use the Bloom Taxonomy to develop a professional development plan for learner assessment and evaluation'

In [1]:
text = "### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?### Assistant:"
tokenizer.add_special_tokens({'pad_token': '<PAD>'})
inputs = tokenizer(text, return_tensors="pt").to(0)
outputs = model.generate(inputs.input_ids,attention_mask=inputs.attention_mask,
                         eos_token_id=tokenizer.eos_token_id,
                         pad_token_id=tokenizer.pad_token_id,
                         max_new_tokens=250,
                         do_sample=False)

print(tokenizer.decode(outputs[0], skip_special_tokens=False))

NameError: name 'tokenizer' is not defined

In [ ]:
tokenizer.pad_token_id

128256

In [ ]:
import gc

def clear_cache():
  torch.cuda.empty_cache()
  gc.collect()

def generate_response(user_message):

  print("\n________________________________________________________________\n")
  print("INPUT:\n"+ user_message)
  print("GENERATION:\n")

  chat = [
    {"role": "system", "content": "You are a helpful assistant, you answer users questions based on your knowledge. responses have to be short and accurate"},
    {"role": "user", "content": user_message},
    # {"role": "user", "content": "I'd like to show off how chat templating works!"},
  ]

  input_ids = tokenizer.apply_chat_template(chat,
                                            add_generation_prompt=True,
                                            return_tensors="pt").to(0)

  # inputs = tokenizer("user: "+ user_message + " Assistant: ", return_tensors="pt").to(0)
  outputs = model.generate( input_ids,
                            # attention_mask=inputs.attention_mask,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.pad_token_id,
                            max_new_tokens=1000,
                            do_sample=True)
  response_txt = tokenizer.decode(outputs[0], skip_special_tokens=False)

  del input_ids
  del outputs

  print("Assistant:\n", response_txt)


In [ ]:
for i in range(2,10):
  if i%2 == 0:
    continue
  generate_response(dataset["train"][i]["prompt"])


________________________________________________________________

INPUT:
bacillus subtilus
GENERATION:

Assistant:
 <|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant, you answer users questions based on your knowledge. responses have to be short and accurate<|eot_id|><|start_header_id|>user<|end_header_id|>

bacillus subtilus<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Bacillus subtilis is a Gram-positive, rod-shaped bacterium commonly found in soil and the gastrointestinal tracts of animals. It's known for its ability to produce antibiotics and is used in various industrial and medical applications.<|eot_id|>

________________________________________________________________

INPUT:
Write long detailed essay about ancient type of religion, totemism
GENERATION:

Assistant:
 <|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant, you answer users questions based on your knowledge. responses have to be s